In [8]:
import sys
sys.path.append("..")
from package import TreeOperator

file_dir = "/home/JYChen/EnergyRegression/samples/DoubleElectron_FlatPT-1to500_13p6TeV/"
IdealPath = f"{file_dir}DoubleElectron_ECALIdealIC_PostEE_124X_30032023.root"
treename = "egRegTree"

reader = TreeOperator()
reader.set_names(inpath=IdealPath, treename=treename)
Idealdf = reader.get_tree(fmt="pd", debug = True)

In [9]:
import json
import pathlib
import shutil

with open("/home/JYChen/EnergyRegression/config/Run3Electron_PostEE.json") as file:
    conf = json.load(file)

In [10]:
def setup_dataset(df, vars, target):
    import pandas as pd
    dataset = pd.DataFrame()
    table = {}
    dataset["isEB"] = df.sc_isEB
    for idx, var in enumerate(vars):
        dataset[f"var{idx}"] = eval(var.format(dfname = "df"))
        table[f"var{idx}"] = var.format(dfname = "").replace(".", "")
    dataset["target"] = eval(target.format(dfname = "df"))
    # dataset.pop("isEB")
    return (table, dataset)

In [11]:
var_eb, dataset_eb = setup_dataset(Idealdf, conf["var_eb"], conf["target1"])
import time
train_dataset = dataset_eb.sample(frac=0.8, random_state=int(time.time()))
test_dataset = dataset_eb.drop(train_dataset.index)

In [14]:
train_dataset = train_dataset[train_dataset["isEB"] == 1]

In [15]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled_input = min_max_scaler.fit_transform(train_dataset.values)

In [ ]:
train_dataset

In [ ]:
scaled_input[0]

In [ ]:
# setup the dataset and the model
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('target')
test_labels = test_features.pop('target')

In [1]:
import torch as th
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from argparse import ArgumentParser
import sys
sys.path.append("..")
from package import PlDataModule, PlDNNModule

def parse_arguments():
    parser = ArgumentParser()

    # program arguments
    parser.add_argument('--checkpoint_name', type=str, default='/home/JYChen/EnergyRegression/log/checkpoint/final.ckpt')
    parser.add_argument('--data_folder', type=str, default='/home/JYChen/EnergyRegression/samples/DoubleElectron_FlatPT-1to500_13p6TeV')
    parser.add_argument('--conf_path', type=str, default='/home/JYChen/EnergyRegression/config/Run3Electron_PostEE.json')
    parser.add_argument('--train_input', type=str, default='DoubleElectron_ECALIdealIC_PostEE_124X_30032023.root')
    parser.add_argument('--test_input', type=str, default='test.csv')
    parser.add_argument('--test_output', type=str, default='test_pred.csv')

    # trainer arguments
    parser.add_argument('--target', type=str, default='{dfname}.mc_energy/{dfname}.sc_rawEnergy')
    parser.add_argument('--default_root_dir', type=str, default='./logs')
    parser.add_argument('--max_epochs', type=int, default=450)
    parser.add_argument('--gpus', type=int, default=(-1 if th.cuda.is_available() else 0))
    parser.add_argument('--auto_select_gpus', type=bool, default=True)
    parser.add_argument('--deterministic', type=bool, default=True)
    parser.add_argument('--batch_size', type=int, default=512)
    parser.add_argument('--num_workers', type=int, default=6)
    parser.add_argument('--learning_rate', type=float, default=0.02)

    # debug parameters - enable fast_dev_run for quick sanity check of the training loop
    parser.add_argument('--fast_dev_run', type=bool, default=False)

    args = parser.parse_args(args=[])
    return args

In [2]:
early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True, mode='min')

In [3]:
def train_model(hparams, region:str):
    
    seed_everything(77)

    ml_module = PlDNNModule(hparams=hparams,)
    data_module = PlDataModule(hparams=hparams, region=region)

    #model_trainer = pl.Trainer.from_argparse_args(hparams)
    # model_trainer = pl.Trainer.add_argparse_args(hparams, callbacks=[early_stop_callback])
    model_trainer = pl.Trainer(callbacks=[early_stop_callback])
    
    model_trainer.logger = pl.loggers.TensorBoardLogger('logs/', name='exp')

    model_trainer.fit(ml_module, data_module)
    model_trainer.save_checkpoint(hparams.checkpoint_name)


# def test_model(hparams):
#     model = PlDNNModule.load_from_checkpoint(hparams.checkpoint_name)
#     output_writer = ResultWriter(model, hparams=hparams)
#     output_writer.test_model()

In [4]:
args = parse_arguments()
train_model(hparams=args, region= "eb")

Seed set to 77
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/JYChen/.conda/envs/plroot/lib/python3.12/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 180 K 
--------------------------------------
180 K     Trainable params
0         Non-trainable params
180 K     Total params
0.723     Total estimated model params size (MB)


(3829, 33)
(3829, 32)
(3829, 1)
(3829, 32)


/home/JYChen/.conda/envs/plroot/lib/python3.12/site-packages/pytorch_lightning/utilities/data.py:104: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
/home/JYChen/.conda/envs/plroot/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/6 [00:00<?, ?it/s] 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/JYChen/.conda/envs/plroot/lib/python3.12/multiprocessing/util.py", line 303, in _run_finalizers
    finalizer()
  File "/home/JYChen/.conda/envs/plroot/lib/python3.12/multiprocessing/util.py", line 303, in _run_finalizers
    finalizer()
  File "/home/JYChen/.conda/envs/plroot/lib/python3.12/multiprocessing/util.py", line 303, in _run_finalizers
    finalizer()
  File "/home/JYChen/.conda/envs/plroot/lib/python3.12/multiprocessing/util.py", line 227, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/JYChen/.conda/envs/plroot/lib/python3.12/multiprocessing/util.py", line 227, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x32 and 10x512)

In [ ]:
def main(args):
    train_model(hparams=args, region= "eb")
    # train_model(hparams=args, region= "ee")
    # test_model(hparams=args)


if __name__ == '__main__':
    args = parse_arguments()
    main(args)

In [10]:
from lightning_utilities.core.imports import RequirementCache

print(RequirementCache("tensorboard"))
print(RequirementCache("tensorboardx"))

DistributionNotFound: The 'tensorboard' distribution was not found and is required by the application. HINT: Try running `pip install -U 'tensorboard'`
DistributionNotFound: The 'tensorboardx' distribution was not found and is required by the application. HINT: Try running `pip install -U 'tensorboardx'`
